In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
def download_30d_single_ticker(
    ticker: str,
    days: int = 30,
    intraday_interval: str = "1m",
    prepost: bool = False,
    outdir: str | None = None,
    fallback_interval: str = "5m",
):
    """
    Returns:
        daily_df: single-level column daily DataFrame (adjusted OHLCV + dividends/splits)
        intraday_dict: {YYYY-MM-DD: single-level column intraday DataFrame}

    Notes:
        1) ticker must be a string (not a list)
        2) multi_level_index=False ensures flat columns
    """
    assert isinstance(ticker, str) and ticker, "ticker must be a non-empty string"

    # --- Daily data: force single-level columns ---
    daily_df = yf.download(
        ticker,
        period=f"{days}d",
        interval="1d",
        auto_adjust=True,
        actions=True,
        progress=False,
        multi_level_index=False,   # important: avoid multi-level columns
    )
    if daily_df.empty:
        raise RuntimeError(f"[{ticker}] Failed to retrieve daily data")

    # --- Intraday data per trading day ---
    intraday_dict: dict[str, pd.DataFrame] = {}
    trade_dates = pd.to_datetime(daily_df.index).tz_localize(None).date

    for d in trade_dates:
        start = pd.Timestamp(d).strftime("%Y-%m-%d")
        end   = (pd.Timestamp(d) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")

        intr = yf.download(
            ticker,
            start=start,
            end=end,
            interval=intraday_interval,
            auto_adjust=True,
            prepost=prepost,
            progress=False,
            multi_level_index=False,
        )
        used = intraday_interval

        if intr.empty and intraday_interval != fallback_interval:
            intr = yf.download(
                ticker,
                start=start,
                end=end,
                interval=fallback_interval,
                auto_adjust=True,
                prepost=prepost,
                progress=False,
                multi_level_index=False,
            )
            used = fallback_interval

        if intr.empty:
            print(f"[Skipped] {ticker} {d} no intraday data available")
            continue

        # Remove timezone info for consistency (plotting/saving)
        if getattr(intr.index, "tz", None) is not None:
            intr = intr.tz_convert(None)

        intr.attrs["interval"] = used
        intr.attrs["prepost"] = prepost
        intraday_dict[str(d)] = intr

    # --- Optional: save to separate files ---
    if outdir:
        # Target directories: data_{TICKER}/daily and data_{TICKER}/intradays
        daily_dir = os.path.join(outdir, "daily")
        intra_dir = os.path.join(outdir, "intradays")
        os.makedirs(daily_dir, exist_ok=True)
        os.makedirs(intra_dir, exist_ok=True)

        # Save daily data
        daily_csv = os.path.join(daily_dir, f"{ticker}_daily_last{days}d.csv")
        daily_df.to_csv(daily_csv)

        # Save intraday data per day
        intraday_files = []
        for d, intr in intraday_dict.items():
            used = intr.attrs.get("interval", "1m")
            intr_csv = os.path.join(intra_dir, f"{ticker}_{d}_intraday_{used}.csv")
            intr.to_csv(intr_csv)
            intraday_files.append(intr_csv)

        print(f"[Saved] Daily data -> {daily_csv}")
        print(f"[Saved] Intraday data ({len(intraday_files)} files) -> {intra_dir}")

    return daily_df, intraday_dict

In [ ]:
TICKER = "SPG"
OUT = f"data_{TICKER}"
daily_df, intradays = download_30d_single_ticker(
    TICKER, days=30, intraday_interval="1m", prepost=False, outdir=OUT
)
print(daily_df.columns.tolist())
any_day = next(iter(intradays)) if intradays else None
if any_day:
    print(any_day, intradays[any_day].columns.tolist(), intradays[any_day].attrs)